<a href="https://colab.research.google.com/github/barbaroja2000/agents/blob/main/hierarchical_crew_ai_exa_anthropic_claude3_groq_mixtral_confluence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autonomous Research Agent - Groq

* Research Agent with Crewai as orchestrator
* Search via:
  * [EXA.ai](https://exa.ai) - Ancedata but seems to generate better quality search output than Tavily - YMMV mind....
  * Arxiv (abstract only, last years papers)
  * Experimental: Wikipedia
* Super fast Groq platfrom / Mixtral 8x7b OR Anthropic models
* Prompts (Agent Backstory & task) finessed using Anthropic [meta-prompt](https://colab.research.google.com/drive/1SoAajN8CBYTl79VyTwxtxncfCWlHlyy9)
* Publishes to Confluence
* Monitoring in Langsmith

In [1]:
!pip install -Uq 'crewai[tools]'  langsmith wikipedia pymupdf arxiv langchain-exa langchain-groq langchain-anthropic atlassian-python-api markdown2 bs4 requests langchain langchain_community groq anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 778.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.6/870.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.6 MB/

In [15]:
research_topic= "Local LLMs"
model_name="claude-3-sonnet-20240229"
manager_model="claude-3-sonnet-20240229"

In [16]:
#@title Imports

import os
from crewai import Agent
import markdown2
import os
import json
from bs4 import BeautifulSoup
import json
from crewai_tools import tool
from exa_py import Exa
from atlassian import Confluence
from langchain_community.utilities import ArxivAPIWrapper

In [17]:
#@title Passwords etc

from google.colab import userdata
exa_api_key=userdata.get('exa_api_key')
groq_api_key=userdata.get('groq_api_key')
anthropic_api_key = userdata.get("anthropic_api_key")

os.environ["CONFLUENCE_API_KEY"]=userdata.get('confluence_api_key')
os.environ["CONFLUENCE_USERNAME"]=userdata.get('confluence_username')
os.environ["CONFLUENCE_URI"]=userdata.get('confluence_uri')

In [18]:
#@title Langsmith
import random
import string

# Generate a random string of 6 characters, including letters and digits
random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=6))

os.environ["LANGCHAIN_API_KEY"] = userdata.get("langchain_api_key")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Autonomous Research Agent - Hierachical"


from langsmith import Client
client = Client()

In [19]:
class PaperFilter:
    def __init__(self):
        self.current_date = datetime.now()
        self.list_strings = []
        self.parsed_papers = []

    def load_papers(self, papers_string):
        self.list_strings = papers_string.split('\n\n')  # Splitting the input string into list of papers
        self.parsed_papers = [self._parse_to_dict(item) for item in self.list_strings]

    def _parse_to_dict(self, list_string):
        summary_index = list_string.find('\nSummary:')
        summary_text = ''
        if summary_index != -1:
            summary_text = list_string[summary_index + 1:]
            list_string = list_string[:summary_index]

        key_value_pairs = list_string.split('\n')
        info_dict = {pair.split(': ')[0]: pair.split(': ')[1] if ': ' in pair else '' for pair in key_value_pairs}

        if summary_text:
            info_dict['Summary'] = summary_text.split(': ', 1)[1]

        return info_dict

    def _is_recent(self, published_date):
        published_datetime = datetime.strptime(published_date, "%Y-%m-%d")
        return (self.current_date - published_datetime) <= timedelta(days=365)

    def _dict_to_string(self, dictionary):
        lines = [f"{key}: {value}" for key, value in dictionary.items()]
        return '\n'.join(lines)

    def filter(self):
        recent_papers = [paper for paper in self.parsed_papers if self._is_recent(paper['Published'])]
        papers_string = "\n\n".join([self._dict_to_string(paper) for paper in recent_papers])
        return papers_string

In [20]:
#@title Tools

import random
import string
from datetime import datetime, timedelta
from langchain_community.utilities import WikipediaAPIWrapper


exa = Exa(api_key=exa_api_key)
paper_filter = PaperFilter()

"""
ARXIV_MAX_QUERY_LENGTH = 300,
load_max_docs = 3,
load_all_available_meta = False,
doc_content_chars_max = 40000
"""

arxiv = ArxivAPIWrapper(top_k_results = 20,  load_max_docs = 20, load_all_available_meta = True, ARXIV_MAX_QUERY_LENGTH = 40000, doc_content_chars_max = 40000)
wikipedia_search = WikipediaAPIWrapper(top_k_results = 5, doc_content_chars_max= 10000)

@tool("Wikipedia search tool")
def search_wikipedia(query: str):
  """Search for a research papers based on the query"""
  docs_string = wikipedia_search.run(query)
  return docs_string


@tool("Arxiv search tool")
def search_arxiv(query: str):
  """Search for a research papers based on the query"""
  docs_string = arxiv.run(query)
  paper_filter.load_papers(docs_string)
  return paper_filter.filter()


@tool("Brave search tool")
def brave_search(query: str):
    """Search for webpages based on s query."""

    return exa.search(f"{query}", use_autoprompt=True, num_results=5)


@tool("EXA search tool")
def search(query: str):
    """Search for a webpage based on the query."""
    return exa.search(f"{query}", use_autoprompt=True, num_results=5)

@tool("EXA similar pages tool")
def find_similar(url: str):
    """Search for webpages similar to a given URL.
    The url passed in should be a URL returned from `search`.
    """
    return exa.find_similar(url, num_results=5)

@tool("EXA get page contents tool")
def get_contents(ids: list[str]):
    """Get the contents of a webpage.
    The ids passed in should be a list of ids returned from `search`.
    """
    return exa.get_contents(ids)

@tool("Confluence Publisher")
def confluence_publisher_tool(content: str) -> str:
    """Use this tool to publish to Confluence"""
    confluence_uri = os.environ["CONFLUENCE_URI"]
    username = os.environ["CONFLUENCE_USERNAME"]
    password = os.environ["CONFLUENCE_API_KEY"]
    space_key = 'AI'

    # Convert Markdown content of the Article to HTML
    html_content = markdown2.markdown(content)

    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the first <h1> tag content for the title
    h1_tag = soup.find('h1')

    page_title = f"{h1_tag.text} - {random_string}" if h1_tag else f"No title found. - {random_string}"

    # Remove the <h1> tag to get the rest of the HTML content without the title
    if h1_tag:
        h1_tag.decompose()

    html_content = str(soup)

    # Initialize Confluence client
    confluence = Confluence(
        url=confluence_uri,
        username=username,
        password=password
    )

    # Get page by title
    parent_info = confluence.get_page_by_title(space_key, "AI Agent Output")

    # Extract the page ID
    parent_id = parent_info['id']

    # Create or update page with HTML content
    response = confluence.create_page(
        space_key,
        page_title,
        html_content,
        parent_id=parent_id,
        type='page',
        representation='storage',
        editor='v2',
        full_width=False
    )

    return json.dumps(response)

In [21]:
#@title Model

#from langchain_groq import ChatGroq
#model = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name=model_name)
#model = ChatAnthropic(model=model_name, anthropic_api_key=anthropic_api_key)

from langchain_anthropic import ChatAnthropic
manager_model = model = ChatAnthropic(model=manager_model, anthropic_api_key=anthropic_api_key)


#from langchain_fireworks import Fireworks

# Initialize a Fireworks model
#manager_model = model = Fireworks(
#    model="accounts/fireworks/models/mixtral-8x22b-instruct-preview",
#    base_url="https://api.fireworks.ai/inference/v1/completions",
#     temperature=0,
#)


In [22]:
#@title Prompts from gists

prompt_dict = {
    "crewai_writer_agent_backstory": "https://gist.githubusercontent.com/barbaroja2000/46d1a4412dd9f8ca6afae2ff87dda346/raw/a061ecaadd54ea096a7679cd33bc906c3fe64b78/crewai_writer_agent_backstory",
    "crewai_researcher_agent_backstory": "https://gist.githubusercontent.com/barbaroja2000/40cd328296296084ef6ede51add117ee/raw/a0f061633a50c8fa30a5750a5b6676297cc7edc2/crewai_researcher_agent_backstory",
     "crewai_writer_editor_agent_backstory": "https://gist.githubusercontent.com/barbaroja2000/eaa413e3862ae202ee650c176a819dd0/raw/6457ad39d26678789e23438c2d16924a5c1beda2/writer_editor_agent_backstory",
    "crewai_research_task" : "https://gist.githubusercontent.com/barbaroja2000/aceab69b70b3a48127695e92d490c881/raw/c24825416a14877732ad6e562aa79d24b48c1ab5/crewai_research_task",
    "crewai_writer_task" : "https://gist.githubusercontent.com/barbaroja2000/304a343fb1780916e480847a53461207/raw/d9654cd2930bdff469808b236e0e38dff352f1d3/crewai_writer_task",
    "crewai_writer_editor_task" : "https://gist.githubusercontent.com/barbaroja2000/c4112a62511782ce84351bc982163dbb/raw/f12166d585fb12faeec3e3a6e661253d43c75f5b/writer_editor_task"
}

In [23]:
#@title Get Prompts

import requests

# Function to download and return Gist content
def download_gist(gist_url):
    try:
        response = requests.get(gist_url)
        response.raise_for_status()  # Raise an error for bad responses
        return response.text
    except requests.RequestException as e:
        print(f"Error downloading Gist: {e}")
        return None

for key, value in prompt_dict.items():
  globals()[key] = download_gist(value)
  print(f"{key}\n", globals()[key])

crewai_writer_agent_backstory
 Expert in tech content strategy with a deep understanding of the tech industry.

Skills:

* Converts complex technical concepts into engaging, easy-to-understand content.
* Expert in structuring articles for readability and engagement.
* Uses subheadings, bullet points, and lists for clarity.
* Applies analogies and real-world examples to demystify technical topics.
* Incorporates visuals like diagrams and infographics to support text.
* Varies sentence structure for dynamic reading.

Signature Style:

* Strong openings and closings that resonate with readers.
* Ability to refine and polish drafts into polished pieces.
* A meticulous attention to detail, aiming to inform, engage, and inspire.

Impact: 

* Transforms technical, dry subjects into compelling reads, making technology accessible to a broader audience.
crewai_researcher_agent_backstory
 I am an advanced AI research assistant, developed by leading experts in the field of artificial intelligence.

In [24]:
#@title Agents

max_iter=10

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Efficiently gather, synthesize, and present the most relevant and accurate information on {topic}',
  verbose=True,
  memory=True,
  max_iter=max_iter,
  backstory=(crewai_researcher_agent_backstory),
  allow_delegation=False,
  llm=model

)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  max_iter=max_iter,
  backstory=(crewai_writer_agent_backstory),
  allow_delegation=False,
  llm=model
)


# Creating a writer agent with custom tools and delegation capability
writer_editor = Agent(
  role='Writer Editor',
  goal='Refine the writers output, suggest areas for improvement',
  verbose=True,
  memory=True,
  max_iter=max_iter,
  backstory=(crewai_writer_editor_agent_backstory),
  allow_delegation=False,
  llm=model
)


# Creating a publisher agent
publisher = Agent(
  role='Publisher',
  goal='Publishes research to Confluence',
  verbose=True,
  memory=True,
  max_iter=max_iter,
  backstory=("I am an AI Publisher. My sole function is to publish finished markdown articles to Confluence."),
  tools=[confluence_publisher_tool],
  allow_delegation=False,
  llm=model
)

In [25]:
#@title Tasks

from crewai import Task
from pydantic import BaseModel

# Research task
research_task = Task(
  description=(crewai_research_task),
  expected_output="A comprehensive 12 paragraph report on {topic}",
  tools=[search, search_arxiv], #, search_wikipedia : this doesnt seem to work great
  agent=researcher,
  output_file=f" {random_string} - research.txt"  # Example of output customization
)

# Writing task with language model configuration
write_task = Task(
  description=(crewai_writer_task),
  expected_output="A comprehensive 8 paragraph article, with 4-5 key points bulleted at the end. All formatted in markdown",
  agent=writer,
  async_execution=False,
  output_file=f" {random_string} - article.md"  # Example of output customization
)


# Writing task with language model configuration
write_edit_task = Task(
  description=(crewai_writer_editor_task),
  expected_output="SUggests for improvements to the writer, for rework",
  agent=writer_editor,
  async_execution=False,
  output_file=f" {random_string} - article.md"  # Example of output customization
)



# Writing task with language model configuration
publish_task = Task(
  description=(
"Publish task to Confluence"
  ),
  expected_output="An article published to confluence.",
  tools=[confluence_publisher_tool],
  agent=publisher,
  async_execution=False,
  output_file=f" {random_string} - .html"  # Example of output customization
)


In [26]:
#@title Crew

from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer, writer_editor, publisher],
  tasks=[research_task, write_task, write_edit_task, publish_task],
  process=Process.hierarchical,
  manager_llm=model
)

In [ ]:
#@title Kick off

# Starting the task execution process with enhanced feedback
result = crew.kickoff({"topic": research_topic, "config" : {"run_name": research_topic}})
print(result)



> Entering new CrewAgentExecutor chain...
Thought: To complete this task, I will need to gather information from authoritative sources on Local LLMs, critically evaluate and synthesize the information, and organize it into a structured report. 

Action: Ask question to co-worker
Action Input: {
    "coworker": "Senior Researcher",
    "question": "Can you provide an overview of Local LLMs, including key concepts, terminology, and an initial breakdown of the core components and areas of focus within this AI subfield?",
    "context": "I need to conduct research on Local Large Language Models (LLMs) for AI. This includes understanding the key concepts, terminology, subfields, core components, and areas of focus related to Local LLMs. An initial high-level overview from an expert would help set the scope for my research."
}


> Entering new CrewAgentExecutor chain...
Thought: Okay, to provide a comprehensive overview of Local Large Language Models (LLMs), I will need to gather and synth